## Tensorflow model

In [782]:
# Import packages for tensorflow
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import metrics
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc

from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, LSTM, LeakyReLU, BatchNormalization, Conv1D, MaxPooling1D, Activation
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import ModelCheckpoint

In [783]:
plt.style.use('fivethirtyeight')
plt.ioff()

In [784]:
var_set = pd.read_csv('var_set.csv', parse_dates=[0],index_col=[0])

In [785]:
var_set

,SalmonPrice,CodPrice,HalibutPrice,CPI,TWI
2013-04-07,39.63,10.754267,47.577612,95.55,92.1500
2013-04-14,41.89,10.800751,47.924958,95.70,92.1920
2013-04-21,43.07,10.774089,47.092639,95.85,92.7980
2013-04-28,42.70,10.569605,47.193296,96.00,93.8080
2013-05-05,41.81,10.605343,48.987917,96.02,93.3125
...,...,...,...,...,...
2022-11-20,82.49,39.417280,76.897925,128.50,122.2180
2022-11-27,79.03,39.071706,73.642666,128.40,122.0320
2022-12-04,78.05,42.701713,73.193629,128.70,120.8780
2022-12-11,75.83,43.652968,74.156841,129.00,122.7060


In [786]:
X = var_set.drop(columns=["SalmonPrice"], axis = 1)
y = var_set["SalmonPrice"]

In [787]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 50, shuffle=False)

In [788]:
x_uni = var_set["SalmonPrice"]

In [789]:
x_uni_train,y_uni_test = train_test_split(x_uni, test_size = 50, shuffle=False)

In [790]:
len(X_train)
y_train.shape

(457,)

In [791]:
# # Make predictions
# test_predictions = []

# first_eval_batch = x_minmax_train[-52:]
# current_batch = first_eval_batch.reshape((1, 52, 1))

# for i in range(len(y_minmax_test)):
#     current_pred = model.predict(current_batch)[0]
#     test_predictions.append(current_pred)
#     current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

# true_predictions = scaler.inverse_transform(test_predictions)

# # Plot the predictions
# test_predictions = pd.Series(true_predictions.reshape(50,))
# y_uni_test = pd.Series(y_uni_test.reshape(50,))
# plt.plot(y_uni_test, label='Test Data')
# plt.plot(test_predictions, label='Predictions')
# plt.legend()

# # Evaluate the model
# from sklearn.metrics import mean_squared_error
# from math import sqrt

# rmse = sqrt(mean_squared_error(y_uni_test, test_predictions))
# print('Test RMSE: %.3f' % rmse)

In [792]:
# Scale x and y multivariate
scaler_x = MinMaxScaler()
X_train = scaler_x.fit_transform(X_train)
X_test = scaler_x.transform(X_test)

# Scale y
scaler_y = MinMaxScaler()
y_train = scaler_y.fit_transform(y_train.values.reshape(-1,1))
y_test = scaler_y.transform(y_test.values.reshape(-1,1))

In [793]:
X_train.shape

(457, 4)

In [794]:
y_train.shape

(457, 1)

In [795]:
#from tensorflow.keras.optimizers.legacy import Adam
batch_size = [1, 2, 13, 26, 52, 104] #i

epochs = [10, 40, 100]
#epochs = [1, 2, 3]

rmse_list = []

n_input = [1, 4, 52, 104] #n
#n_input = [24, 52] #n

run = 0
b = 0
e = 0

m1 = 0
m2 = 0

uni_multi = [X_train, y_train]

optimuzer = ['adam', 'nadam']

rmse_df = pd.DataFrame(columns=['Run', 'Batch Size', 'Epochs'])
for i in batch_size:
    for opt in optimuzer:
        for n in n_input:
            b = b + 1
            for j in epochs:
                for m1 in range(1):
                    for vari in uni_multi:
                        e = e + 1
                        print('Batch size: '+str(i))
                        print('Lookback -- input: '+str(n))
                        print('Optimizer: '+str(opt))
                        print('Epochs: '+str(j))
                        
                        if run % 2 == 0:
                            uni_name = 'Univariate'
                        else:
                            uni_name = 'Multivariate'
                        print(uni_name)
                        run = run + 1
                        print('Run: '+str(run))
                        generated_batches = TimeseriesGenerator(vari, y_train, length=n, batch_size=1)
                        model_name_string = 'Model'+ ' ' + str(run)
                        model_name = 'model_name' + str(run)
                        model_name_test = 'model_name' + str(run)
                        print(model_name)
                        model_name = Sequential()
                        model_name.add(Flatten(input_shape=(n,1)))
                        #model_name.add(Dense(1024, activation='relu', name="hidden_layer_1"))
                        if m1 == 1:
                            model_name.add(Dense(128, activation='relu'))
                            model_name.add(Dropout(0.1))
                            model_name.add(Dense(64, activation='relu'))
                            model_name.add(Dropout(0.1))
                        #m1 = m1 + 1
                        model_name.add(Dense(32, activation='relu'))
                        model_name.add(Dense(1))
                        model_name.compile(optimizer=opt, loss='mse')
                        history = 'history'+str(run)
                        history = model_name.fit(generated_batches,epochs=j,verbose=0, batch_size=i)
                        #print(model_name.summary())
                        model_name_test = model_name
                        test_predictions = []
                        first_eval_batch = y_train[-n:]
                        current_batch = first_eval_batch.reshape((1, n, 1))
                        print(current_batch)
                        for k in range(len(y_test)):
                            current_pred = model_name.predict(current_batch)[0]
                            test_predictions.append(current_pred)
                            current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)
                        true_predictions = scaler_y.inverse_transform(test_predictions)
                        #print(test_predictions)
                        
                        y_test_inverse = scaler_y.inverse_transform(y_test)
                        rmse = sqrt(mean_squared_error(y_test_inverse, true_predictions))
                        rmse_df = rmse_df.append({'Model Name' : model_name_string, 'Run': run, 'Model alternative':m1+1, 'Batch Size': i, 'Epochs': j, 'Optimizer' : opt, 'Time steps':n, 'Uni/Multi' :uni_name, 'RMSE': rmse}, ignore_index=True)
                        rmse_list.append(rmse)
                        print('Test RMSE: %.3f' % rmse)
                        #mse = mean_squared_error(y_test_inverse, true_predictions)
                        #print('Test MSE: %.3f' % mse)
                        plot_predictions = pd.Series(true_predictions.reshape(50,))
                        y_test_inverse = pd.Series(y_test_inverse.reshape(50,))
                        plt.figure(figsize=(10, 6))
                        plt.plot(history.history['loss'], label='Loss(MSE)')
                        plt.legend()
                        plt.savefig('Figures/Neural networks/ForLoop_Tensor/plotLoss_'+str(run)+'.png')

                        plt.figure(figsize=(18,10))
                        plt.plot(plot_predictions, label='Forecast')
                        plt.plot(y_test_inverse, label='Actual')
                        plt.legend()
                        plt.savefig('Figures/Neural networks/ForLoop_Tensor/plotModel_'+str(run)+'.png')

Batch size: 1
Lookback -- input: 1
Optimizer: adam
Epochs: 10
Univariate
Run: 1
model_name1


2023-04-25 17:24:29.510112: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


[[[0.82496224]]]
1/1 [==============================] - 0s 11ms/step


2023-04-25 17:24:49.687788: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 11ms/step
Test RMSE: 32.274
Batch size: 1
Lookback -- input: 1
Optimizer: adam
Epochs: 10
Multivariate
Run: 2
model_name2


...

[[[0.82496224]]]
1/1 [==============================] - 0s 13ms/step


2023-04-25 17:25:09.119682: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 11ms/step
Test RMSE: 31.126
Batch size: 1
Lookback -- input: 1
Optimizer: adam
Epochs: 40
Univariate
Run: 3
model_name3


/var/folders/xy/_0zst1c12b9_m_438bw49zw40000gn/T/ipykernel_74781/3651584601.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rmse_df = rmse_df.append({'Model Name' : model_name_string, 'Run': run, 'Model alternative':m1+1, 'Batch Size': i, 'Epochs': j, 'Optimizer' : opt, 'Time steps':n, 'Uni/Multi' :uni_name, 'RMSE': rmse}, ignore_index=True)
2023-04-25 17:25:10.857091: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 

In [796]:
# y_test = scaler_y.inverse_transform(y_test)
# plot_predictions = pd.Series(true_predictions.reshape(50,))
# y_test = pd.Series(y_test.reshape(50,))
# plt.plot(y_test, label='Test Data')
# plt.plot(plot_predictions, label='Predictions')
# plt.legend()

In [797]:
rmse_list

[32.27404859161346, 31.126496687922444]

In [798]:
true_predictions

array([[70.45275782],
       [69.40175352],
       [68.40413469],
       [67.45718976],
       [66.55833973],
       [65.7051445 ],
       [64.89529024],
       [64.12657044],
       [63.39689224],
       [62.70427957],
       [62.04684789],
       [61.42281054],
       [60.83046923],
       [60.26821722],
       [59.73452038],
       [59.22793295],
       [58.74706916],
       [58.29050535],
       [57.85701359],
       [57.44543221],
       [57.05465324],
       [56.68362235],
       [56.33134203],
       [55.99686528],
       [55.67929246],
       [55.37777126],
       [55.09148727],
       [54.81966711],
       [54.5615879 ],
       [54.31654727],
       [54.08389182],
       [53.86299492],
       [53.65325999],
       [53.45412512],
       [53.2650537 ],
       [53.08553906],
       [52.91509668],
       [52.7532704 ],
       [52.59961988],
       [52.45373475],
       [52.31522357],
       [52.18371072],
       [52.0588458 ],
       [51.94029263],
       [51.82772922],
       [51

In [799]:
y_test_inverse

0      71.52
1      70.81
2      73.48
3      80.66
4      92.82
5      91.41
6      85.35
7      82.96
8      80.61
9      86.38
10     88.64
11    102.78
12    109.91
13    115.17
14    125.87
15    122.75
16    104.69
17     99.34
18    102.89
19    101.13
20    107.09
21    106.90
22    105.15
23     98.30
24     95.84
25     92.30
26     79.14
27     70.89
28     72.56
29     71.69
30     65.78
31     59.56
32     57.15
33     60.74
34     62.40
35     62.88
36     63.23
37     68.21
38     71.80
39     70.86
40     69.96
41     67.62
42     66.20
43     70.70
44     80.64
45     82.49
46     79.03
47     78.05
48     75.83
49     86.64
dtype: float64

In [800]:
rmse_df['Epochs'] = rmse_df['Epochs'].astype(int)
rmse_df['Batch Size'] = rmse_df['Batch Size'].astype(int)
rmse_df['Run'] = rmse_df['Run'].astype(int)
rmse_df['Model alternative'] = rmse_df['Model alternative'].astype(int)
rmse_df['Time steps'] = rmse_df['Time steps'].astype(int)

In [801]:
rmse_df.insert(8, 'RMSE', rmse_df.pop('RMSE'))

In [802]:
rmse_df

,Run,Batch Size,Epochs,Model Name,Model alternative,Optimizer,Time steps,Uni/Multi,RMSE
0,1,1,10,Model 1,1,adam,1,Univariate,32.274049
1,2,1,10,Model 2,1,adam,1,Multivariate,31.126497


In [803]:
rmse_df.style.to_latex("Figures/Neural networks/RMSE_Tensorflow.tex")

In [804]:
rmse_df.to_csv('Figures/Neural networks/RMSE_Tensorflow.csv')

In [805]:
rmse_df.sort_values(by=['RMSE']).head(20).style.to_latex("Figures/Neural networks/RMSE_Tensorflow_top20.tex")

In [806]:
# pred_list = []

# batch = X_train[-n_input:].reshape((1, n_input, 1))

# for i in range(n_input):   
#     pred_list.append(model_name.predict(batch)[0]) 
#     batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)

In [807]:
# pred_list = scaler_x.inverse_transform(pred_list)

In [808]:
# plt.plot(pred_list, label='Predictions')
# plt.plot(scaler_x.inverse_transform(X_test), label='True Data')

In [809]:
# from pandas.tseries.offsets import DateOffset
# df = pd.DataFrame()
# add_dates = [df.index[-1] + DateOffset(months=x) for x in range(0,13) ]
# future_dates = pd.DataFrame(index=add_dates[1:],columns=df.columns)

In [810]:
# rmse_df.sort_values(by=['RMSE'], ascending=True)

In [811]:
# Create dataframe with RMSE values when length of values does not match length of index
# rmse_df = pd.DataFrame(rmse_list, columns = ['RMSE'])


In [812]:
# rmse_df

In [813]:
# # Create dataframe with RMSE values
# rmse_df = pd.DataFrame(rmse_list, columns = ['RMSE'])
# rmse_df['Batch Size'] = batch_size
# rmse_df['Epochs'] = epochs

# # Plot RMSE values
# plt.plot(rmse_df['RMSE'], label='RMSE')
# plt.legend()

# # Find the best model_name
# rmse_df[rmse_df['RMSE'] == rmse_df['RMSE'].min()]